In [1]:
import numpy as np

# quara
from quara.objects.matrix_basis import (
    get_normalized_pauli_basis,
)
from quara.objects.elemental_system import ElementalSystem
from quara.objects.composite_system import CompositeSystem
from quara.objects.composite_system_typical import generate_composite_system
from quara.objects.tester_typical import (
    generate_tester_states,
    generate_tester_povms,
)
from quara.objects.qoperation_typical import generate_qoperation
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.standard_qpt import StandardQpt

from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.loss_minimization_estimator import (
    LossMinimizationEstimator,
)
from quara.loss_function.weighted_probability_based_squared_error import (
    WeightedProbabilityBasedSquaredError,
    WeightedProbabilityBasedSquaredErrorOption,
)
from quara.minimization_algorithm.projected_gradient_descent_backtracking import (
    ProjectedGradientDescentBacktracking,
    ProjectedGradientDescentBacktrackingOption,
)

In [2]:
# 1-qubit Quantum State Tomography

In [3]:
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)
print(c_sys)

elemental_systems:
[0] 0 (system_id=4455023280)

dim: 2
basis:
(array([[0.70710678+0.j, 0.        +0.j],
       [0.        +0.j, 0.70710678+0.j]]), array([[0.        +0.j, 0.70710678+0.j],
       [0.70710678+0.j, 0.        +0.j]]), array([[0.+0.j        , 0.-0.70710678j],
       [0.+0.70710678j, 0.+0.j        ]]), array([[ 0.70710678+0.j,  0.        +0.j],
       [ 0.        +0.j, -0.70710678+0.j]]))


In [4]:
# Testers
names = ["x", "y", "z"]
testers = generate_tester_povms(c_sys=c_sys, names = names)
for i, tester in enumerate(testers):
    print("Tester ", i, ":\n", tester)

Tester  0 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.70710678  0.          0.        ]
 [ 0.70710678 -0.70710678  0.          0.        ]]
Tester  1 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.70710678  0.         -0.70710678  0.        ]]
Tester  2 :
 Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]
/Users/takanori/Work/dev/quara/quara/utils/matrix_util.py:195: ComplexWarning: Casting complex values to real discards the imaginary part
  tmp_hs = tmp_hs.astype(np.float64)


In [5]:
# Quantum State Tomography
seed = 7896
qst = StandardQst(testers, on_para_eq_constraint=True, schedules="all", seed=seed)

In [6]:
mode = "state"
name = "a"
true = generate_qoperation(mode=mode, name=name, c_sys=c_sys)
print(true)

Type:
State

Dim:
2

Vec:
[0.70710678 0.5        0.5        0.        ]


In [7]:
prob_dists = qst.calc_prob_dists(true)
print(prob_dists)

[[0.85355339 0.14644661]
 [0.85355339 0.14644661]
 [0.5        0.5       ]]


In [27]:
num_data = 1000
empi_dists = qst.generate_empi_dists(state=true, num_sum=num_data)
for f in empi_dists:
    print(f)

(1000, array([0.852, 0.148]))
(1000, array([0.855, 0.145]))
(1000, array([0.465, 0.535]))


In [28]:
estimator = LinearEstimator()
result = estimator.calc_estimate(qtomography=qst, empi_dists=empi_dists, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("is estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are: ", estimate.calc_eigenvalues())

Type:
State

Dim:
2

Vec:
[ 0.70710678  0.49780317  0.50204581 -0.04949747]
is estimate physical? :  False

Eigenvalues are:  [1.0011526713487617, -0.0011526713487617635]


In [29]:
estimator = LossMinimizationEstimator()
loss = WeightedProbabilityBasedSquaredError(3)
loss_option = WeightedProbabilityBasedSquaredErrorOption("identity")
algo = ProjectedGradientDescentBacktracking()

obj_start = (qst.generate_empty_estimation_obj_with_setting_info().generate_origin_obj())
var_start = obj_start.to_var()
algo_option = ProjectedGradientDescentBacktrackingOption(mode_stopping_criterion_gradient_descent="sum_absolute_difference_variable", num_history_stopping_criterion_gradient_descent=1)

result = estimator.calc_estimate(qtomography=qst, empi_dists=empi_dists, loss=loss, loss_option=loss_option, algo=algo, algo_option=algo_option, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are: ", estimate.calc_eigenvalues())

Type:
State

Dim:
2

Vec:
[ 0.70710678  0.49665823  0.50089111 -0.04938363]

is estimate physical? :  False

Eigenvalues are:  [1.0000000203091033, -2.030910339900771e-08]


In [30]:
# POVM Tomography

# Composite System
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)
#print(c_sys)

# Testers
names = ["x0", "y0", "z0", "z1"]
testers = generate_tester_states(c_sys=c_sys, names = names)
povmt = StandardPovmt(testers, num_outcomes=2, on_para_eq_constraint=True, schedules="all")

/Users/takanori/Work/dev/quara/quara/utils/matrix_util.py:195: ComplexWarning: Casting complex values to real discards the imaginary part
  tmp_hs = tmp_hs.astype(np.float64)


In [31]:
mode = "povm"
name = "z"
true = generate_qoperation(mode=mode, name=name, c_sys=c_sys)
print(true)

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678  0.          0.          0.70710678]
 [ 0.70710678  0.          0.         -0.70710678]]


In [32]:
prob_dists = povmt.calc_prob_dists(true)
print(prob_dists)

[[5.00000000e-01 5.00000000e-01]
 [5.00000000e-01 5.00000000e-01]
 [1.00000000e+00 2.22044605e-16]
 [0.00000000e+00 1.00000000e+00]]


In [33]:
num_data = 1000
empi_dists = povmt.generate_empi_dists_sequence(povm=true, num_sums=[num_data])[0]
for f in empi_dists:
    print(f)

(1000, array([0.469, 0.531]))
(1000, array([0.52, 0.48]))
(1000, array([1., 0.]))
(1000, array([0., 1.]))


In [36]:
estimator = LinearEstimator()
result = estimator.calc_estimate(qtomography=povmt, empi_dists=empi_dists, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are:", estimate.calc_eigenvalues())

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.70710678 -0.04384062  0.02828427  0.70710678]
 [ 0.70710678  0.04384062 -0.02828427 -0.70710678]]

is estimate physical? :  False

Eigenvalues are: [[1.0013591527039272, -0.0013591527039275215], [1.001359152703927, -0.0013591527039273005]]


In [37]:
estimator = LossMinimizationEstimator()
loss = WeightedProbabilityBasedSquaredError(4)
loss_option = WeightedProbabilityBasedSquaredErrorOption("identity")
algo = ProjectedGradientDescentBacktracking()  
algo_option = ProjectedGradientDescentBacktrackingOption()

result = estimator.calc_estimate(qtomography=povmt, empi_dists=empi_dists, loss=loss, loss_option=loss_option, algo=algo, algo_option=algo_option, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
print("\nEigenvalues are:", estimate.calc_eigenvalues())

Type:
Povm

Dim:
2

Number of outcomes:
2

Vecs:
[[ 0.7071068  -0.04360487  0.02813213  0.70520013]
 [ 0.70710676  0.04360487 -0.02813213 -0.70520013]]

is estimate physical? :  False

Eigenvalues are: [[1.0000000232972743, -1.8865117801247777e-17], [1.0, -2.3297274350572023e-08]]


In [38]:
# Quantum Process Tomography

# Composite System
mode = "qubit"
num = 1
c_sys = generate_composite_system(mode=mode, num=num)
#print(c_sys)

# Testers
names_states = ["x0", "y0", "z0", "z1"]
testers_states = generate_tester_states(c_sys=c_sys, names = names_states)
names_povms = ["x", "y", "z"]
testers_povms = generate_tester_povms(c_sys=c_sys, names=names_povms)
qpt = StandardQpt(states=testers_states, povms=testers_povms, on_para_eq_constraint=True, schedules="all")

/Users/takanori/Work/dev/quara/quara/utils/matrix_util.py:195: ComplexWarning: Casting complex values to real discards the imaginary part
  tmp_hs = tmp_hs.astype(np.float64)


In [39]:
mode = "gate"
name = "hadamard"
true = generate_qoperation(mode=mode, name=name, c_sys=c_sys)
print(true)

Type:
Gate

Dim:
2

HS:
[[ 1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0. -1.  0.]
 [ 0.  1.  0.  0.]]


In [40]:
prob_dists = qpt.calc_prob_dists(true)
print(prob_dists)

[[0.5 0.5]
 [0.5 0.5]
 [1.  0. ]
 [0.5 0.5]
 [0.  1. ]
 [0.5 0.5]
 [1.  0. ]
 [0.5 0.5]
 [0.5 0.5]
 [0.  1. ]
 [0.5 0.5]
 [0.5 0.5]]


In [41]:
num_data = 1000
empi_dists = qpt.generate_empi_dists_sequence(gate=true, num_sums=[num_data])[0]
for f in empi_dists:
    print(f)

(1000, array([0.503, 0.497]))
(1000, array([0.515, 0.485]))
(1000, array([1., 0.]))
(1000, array([0.489, 0.511]))
(1000, array([0., 1.]))
(1000, array([0.517, 0.483]))
(1000, array([1., 0.]))
(1000, array([0.495, 0.505]))
(1000, array([0.509, 0.491]))
(1000, array([0., 1.]))
(1000, array([0.528, 0.472]))
(1000, array([0.472, 0.528]))


In [43]:
estimator = LinearEstimator()
result = estimator.calc_estimate(qtomography=qpt, empi_dists=empi_dists, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
evals, evecs = np.linalg.eigh(estimate.to_choi_matrix())
print("\nEigenvalues are:", evals)

Type:
Gate

Dim:
2

HS:
[[ 1.     0.     0.     0.   ]
 [ 0.     0.006 -0.022  1.   ]
 [ 0.023  0.007 -1.023 -0.033]
 [-0.019  1.019  0.053  0.037]]

is estimate physical? :  False

Eigenvalues are: [-0.04409214 -0.00839776  0.03044277  2.02204713]


In [44]:
estimator = LossMinimizationEstimator()
loss = WeightedProbabilityBasedSquaredError(12)
loss_option = WeightedProbabilityBasedSquaredErrorOption("identity")
algo = ProjectedGradientDescentBacktracking()  
algo_option = ProjectedGradientDescentBacktrackingOption()

result = estimator.calc_estimate(qtomography=qpt, empi_dists=empi_dists, loss=loss, loss_option=loss_option, algo=algo, algo_option=algo_option, is_computation_time_required=True)
estimate = result.estimated_qoperation
print(estimate)
print("\nis estimate physical? : ", estimate.is_physical())
evals, evecs = np.linalg.eigh(estimate.to_choi_matrix())
print("\nEigenvalues are:", evals)

Type:
Gate

Dim:
2

HS:
[[ 1.          0.          0.          0.        ]
 [ 0.00587047 -0.01815819 -0.02900601  0.99279653]
 [ 0.00477508  0.03162411 -0.98935507 -0.03218338]
 [-0.00463472  0.99237513  0.03188348  0.02742128]]

is estimate physical? :  False

Eigenvalues are: [-1.98972660e-08 -1.32440433e-08  1.14832271e-02  1.98851681e+00]
